In [1]:
import getpass
user_id = getpass.getuser()

import sys
sys.path.append(f"/home/{user_id}/OTTEHR/")
sys.path.append(f"/home/{user_id}/Statistical-Fairness-Test")

from collections import Counter
from common import *
from googletrans import Translator
import matplotlib.pyplot as plt
from mimic_common import *
from sklearn.decomposition import PCA

base_path = f"/home/{user_id}/OTTEHR/mimiciv_exp"
output_path = f"/home/{user_id}/OTTEHR/outputs/mimiciv"

2024-02-28 15:24:01.886819: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 15:24:01.924337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 15:24:01.924378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 15:24:01.925503: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 15:24:01.931843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

The goal is to run Taskesen's statistical test to study the relationship between explanatory variable ICD code and label Insolin prescription based on protected attribute race (black and white)

Step 1: Select source and target samples

In [2]:
df = pd.read_csv(os.path.join(base_path, "mimiciv/final_admission.csv"), converters={'diagnosis_icd_code': literal_eval, 'procedure_icd_code': literal_eval, 'drug': literal_eval})
display(df)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,diagnosis_icd_code,procedure_icd_code,drug
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,2180-05-06 23:30:00,0,F,52,2180,2014 - 2016,2180-09-09,"[5723, 78959, 5715, 07070, 496, 29680, 30981, ...",[5491],"[Acetaminophen, Sodium Chloride 0.9% Flush, F..."
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,...,2180-06-26 21:31:00,0,F,52,2180,2014 - 2016,2180-09-09,"[07071, 78959, 2875, 2761, 496, 5715, V08, 3051]",[5491],"[Rifaximin, Lactulose, Calcium Carbonate, Furo..."
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,...,2180-08-06 01:44:00,0,F,52,2180,2014 - 2016,2180-09-09,"[07054, 78959, V462, 5715, 2767, 2761, 496, V0...",[5491],"[Albumin 25% (12.5g / 50mL), Calcium Carbonate..."
3,10000117,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaN,OBSERVATION ADMIT,P75S70,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Other,...,2183-09-18 20:20:00,0,F,48,2174,2008 - 2010,NaN,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,...",[0QS734Z],"[Senna, Bisacodyl, Cepacol (Sore Throat Lozeng..."
4,10000560,28979390,2189-10-15 10:30:00,2189-10-17 15:00:00,NaN,SURGICAL SAME DAY ADMISSION,P08ZLF,PHYSICIAN REFERRAL,HOME,Other,...,NaN,0,F,53,2189,2008 - 2010,NaN,"[1890, V1582, V1201]",[5551],"[Scopolamine Patch, Ketorolac, Lorazepam, Pota..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220848,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,2149-01-08 18:12:00,0,F,46,2147,2017 - 2019,NaN,"[T8141XA, E1110, K632, D682, L02211, T83728A, ...","[0J980ZZ, 0WPF0JZ, 05HY33Z]","[Insulin, Potassium Chloride, Insulin, Insulin..."
220849,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,...,2147-07-18 17:34:00,0,F,46,2147,2017 - 2019,NaN,"[T8131XA, T8183XA, K632, D6851, N390, Y838, Y9...","[0HR7X74, 0HBJXZZ, 0HBHXZZ, 02HV33Z, 3E0436Z]","[Insulin, Potassium Chloride (Powder), Acetami..."
220850,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,...,2164-09-10 14:46:00,1,M,58,2164,2008 - 2010,2164-09-17,"[3453, 51881, 5070, 5180, 42741, 43821, 43811,...","[9604, 9672, 966, 0331, 0331]","[Potassium Chloride, Bisacodyl, Phenytoin, 0.9..."
220851,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaN,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,...,2164-07-25 01:20:00,0,M,58,2164,2008 - 2010,2164-09-17,"[43491, 43820, 34590, 43811, 4019, 2724, 3051]","[8891, 8841]","[Metoprolol Tartrate, Sodium Chloride 0.9% Fl..."


In [3]:
def select_drug_samples(df, group_name, drug, source, target, source_count = None, target_count = None):
    """ 
    Select rows in the dataframe df for source and target samples, return the results in \
        a format that is compatible to Taskesen et al's statistical test 
    
    :param df Dataframe: the dataframe to select samples from.
    :param str drug: drug name, used for preparing the labels array
    :param str source: source group name, e.g., White
    :param str target: target group name, e.g., Black
    :param int source_count: if specified, the number of samples for the source. \
        Otherwise, delete all rows that do not belong to target or source
    :param int target_count: if specified, the number of samples for the target. \
        Otherwise, delete all rows that do not belong to target or source
    """
    df_copy = copy.deepcopy(df)
    
    source_indices = []
    target_indices = []
    other_indices = []
    for index, row in df_copy.iterrows():
        if row[group_name] == source:
            source_indices.append(index)
        elif row[group_name] == target:
            target_indices.append(index)
        else:
            other_indices.append(index)

    
    # indices to delete from the dataframe
    if source_count is not None:
        delete_source_indices = random.sample(source_indices, len(source_indices)-source_count)
        delete_target_indices = random.sample(target_indices, len(target_indices)-target_count)

        other_indices.extend(delete_source_indices)
        other_indices.extend(delete_target_indices)

    df_copy = df_copy.drop(other_indices, axis=0, inplace=False)

    # prepare labels array, indicator of label attributes and \
    # sensitive_labels array: indicator of protected attributes
    labels = []
    sensitive_labels = []
    for _, row in df_copy.iterrows():
        if drug in row['drug']:
            labels.append(1)
        else:
            labels.append(0)

        if row[group_name] == source:
            sensitive_labels.append(1)
        elif row[group_name] == target:
            sensitive_labels.append(0)

    return df_copy, np.array(labels), np.array(sensitive_labels)

In [4]:
source_num = 100
target_num = 100
selected_df, labels, sensitive_labels = select_drug_samples(df, 'race', 'Insulin', 'WHITE', 'ASIAN', source_num, target_num)

In [5]:
display(selected_df)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,diagnosis_icd_code,procedure_icd_code,drug
698,10031816,23514622,2195-12-10 12:00:00,2195-12-11 17:00:00,NaN,SURGICAL SAME DAY ADMISSION,P29BE6,PHYSICIAN REFERRAL,HOME,Other,...,NaN,0,M,54,2194,2008 - 2010,2200-07-22,"[7892, V1005]",[415],"[Docusate Sodium, Heparin, Acetaminophen, Oxyc..."
1411,10069620,26705728,2161-10-25 14:42:00,2161-10-26 15:45:00,NaN,EU OBSERVATION,P612YK,EMERGENCY ROOM,NaN,Other,...,2161-10-25 15:58:00,0,M,45,2161,2008 - 2010,NaN,"[5110, 4928, 71536, 7242, 3051]","[8952, 8938, 8744, 8847, 8843, 3899]","[Ibuprofen, OxycoDONE (Immediate Release) , Mo..."
2083,10102610,24362638,2173-12-03 11:39:00,2173-12-08 15:03:00,NaN,DIRECT EMER.,P45GUA,PHYSICIAN REFERRAL,HOME,Other,...,NaN,0,F,24,2173,2017 - 2019,NaN,"[Z5111, C8529, I82290, R740, T451X5A, Y929, G6...","[3E04305, 0JH60WZ, 02H633Z]","[Bisacodyl, LORazepam, PredniSONE, Docusate So..."
2090,10103031,27157489,2189-03-19 03:23:00,2189-03-21 16:00:00,NaN,URGENT,P75S70,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,...,2189-03-19 04:20:00,0,M,61,2189,2014 - 2016,NaN,"[82322, 30000, 8240, V08, V0481, 311, E8849, E...",[7936],"[Fluoxetine, Omeprazole, Emtricitabine-Tenofov..."
3921,10183015,29059579,2139-09-27 04:58:00,2139-10-01 17:40:00,NaN,EW EMER.,P30SZF,EMERGENCY ROOM,HOME,Other,...,2139-09-27 07:25:00,0,F,26,2137,2008 - 2010,NaN,"[78906, 7291, 78791, 53081, 5533, 311, 3379, 5...","[4516, 4525, 4513]","[OxycoDONE (Immediate Release) , Ranitidine, L..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217098,19827577,24329975,2115-10-07 02:02:00,2115-10-09 17:45:00,NaN,OBSERVATION ADMIT,P27SDD,EMERGENCY ROOM,HOME,Medicare,...,2115-10-07 18:40:00,0,F,80,2115,2008 - 2010,NaN,"[78659, 5849, 27651, 40390, 5859, 25050, 36201...","[3722, 8856, 8845]","[Insulin, Metoprolol Tartrate, Carvedilol, Pot..."
217588,19854456,27567690,2177-10-27 20:56:00,2177-11-04 14:48:00,NaN,URGENT,P29CGZ,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,...,NaN,0,M,87,2177,2008 - 2010,2178-05-30,"[1531, 56089, 1977, 1985, 185, 2724, 42731, 24...","[4685, 966]","[Levothyroxine Sodium, Ondansetron ODT, Hepari..."
218598,19899322,21898875,2157-03-01 17:11:00,2157-03-04 18:00:00,NaN,URGENT,P779L2,TRANSFER FROM HOSPITAL,HOME,Other,...,NaN,0,F,77,2157,2017 - 2019,2157-04-07,"[C251, I82890, C786, R188, I10, I868, L271, T4...",[0FBG8ZX],"[Potassium Chloride, Polyethylene Glycol, Morp..."
219116,19920914,29344589,2135-08-30 18:34:00,2135-09-14 17:45:00,NaN,URGENT,P3529J,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,...,NaN,0,F,72,2134,2011 - 2013,2135-10-09,"[4592, 5119, 2762, 2875, 1629, 42832, 4280, 27...","[3950, 3990, 0045, 0040, 8851, 3404, 3491, 922...","[Sodium Chloride 0.9%, Furosemide, 5% Dextrose..."


In [6]:
Counter(labels)

Counter({0: 143, 1: 57})

In [7]:
Counter(sensitive_labels)

Counter({0: 100, 1: 100})

In [8]:
def gen_feature(df, feature_code_name):
    """ 
    Generate source features and target features from dataframe df
    :param str feature_code_name: name of the input. For the experiments on ICD codes vs. duration, the input_name is 'ICD codes'

    """
    unique_code_dict, num_codes = find_unique_code(df, ICD_name = feature_code_name)

    # Prepare source
    features = np.empty(shape=[df.shape[0], num_codes])
    feature_index = 0
    for _, row in df.iterrows():
        code_ind = np.zeros(num_codes)
        for code in row[feature_code_name]:
            code_ind[unique_code_dict[code]] += 1
        features[feature_index] = code_ind
        feature_index += 1

    return features

In [9]:
features = gen_feature(selected_df, 'diagnosis_icd_code')

In [10]:
pca = PCA(n_components=50)
reps = pca.fit_transform(features)
reps = features

In [11]:
print(features.shape, reps.shape)

(200, 1190) (200, 1190)


Step 2: Run Taskesen's statistical test

In [12]:
from load_data import *
from matplotlib import colors as mcolors
from random import seed
from functions import *
import time
from LogisticRegression import LogisticRegression
import datetime
from scipy.stats import gamma

In [13]:
clf = LogisticRegression(fit_intercept=False)
clf.fit(X=reps, y=labels)
beta = clf.coef_
print(beta)

[-0.22073351 -0.26645922  1.37786125 ... -6.33363682  0.86820034
  0.07243923]


/home/wanxinli/miniconda3/lib/python3.9/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [14]:
data_tuple = [reps, np.array(sensitive_labels), np.array(labels)]
range_gamma = [-5, 5]
range_of_k = np.linspace(0, 1/8, 100)
tol = 1e-10

In [15]:
test_marginals = np.reshape(get_marginals(sensitives=sensitive_labels, target=labels), [2, 2])
c = limiting_dist_EQOPP(X=reps, a=sensitive_labels, y=labels, beta=beta, marginals=test_marginals)
theta = 2 * c
sig_value = gamma.ppf(.95, a=1/2, scale=theta)
print(f"The rejection significance value is {sig_value}")

The rejection significance value is 5.418572142936755e-07


In [16]:
dist, optimum_ks, gamma_opt = calculate_distance_prob_eqopp_with_opt_params(data_tuple=data_tuple,
                                          function_of_gamma=f_gamma,
                                          range_gamma=range_gamma,
                                          k_opt_fcn=k_opt,
                                          range_of_k=range_of_k,
                                          beta=beta, tol=tol)

gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.0, beta.T @ x_hat is 15.926870417147004, the term within exp is: -15.926870417147004\gamma is -1.1803398874989481, lambd is -7.4074074074074066, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.0, beta.T @ x_hat is 15.90951176738799, the term within exp is: -15.90951176738799\gamma is -1.1803398874989481, lambd is -7.4074074074074066, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.0, beta.T @ x_hat is 15.941442019734199, the term within exp is: -15.941442019734199\gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.0, beta.T @ x_hat is 15.94294914008392, the term within exp is: -15.94294914008392\gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.0, beta.T @ x_hat is 16.037705615386955, the term within exp is: -16.037705615386955\gamma is -1.1803398874989481, lambd

/home/wanxinli/Statistical-Fairness-Test/functions.py:52: RuntimeWarning: overflow encountered in exp
  gamma * lambd / (1 + np.exp(gamma * lambd * LA.norm(beta, ord=2) ** 2 * k - beta.T @ x_hat))) # objective function



gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.011363636363636364, beta.T @ x_hat is 15.926870417147004, the term within exp is: -681.0921326089826\



/home/wanxinli/Statistical-Fairness-Test/functions.py:52: RuntimeWarning: overflow encountered in exp
  gamma * lambd / (1 + np.exp(gamma * lambd * LA.norm(beta, ord=2) ** 2 * k - beta.T @ x_hat))) # objective function


gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.010101010101010102, beta.T @ x_hat is 15.929220739413186, the term within exp is: -607.1872315766004\gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.011363636363636364, beta.T @ x_hat is 15.94294914008392, the term within exp is: -681.1082113319195\

gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.010101010101010102, beta.T @ x_hat is 15.911195270372076, the term within exp is: -607.1692061075594\

gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.010101010101010102, beta.T @ x_hat is 15.907704177439244, the term within exp is: -607.1657150146265\

...

gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.011363636363636364, beta.T @ x_hat is 15.92267841889304, the term within exp is: -681.0879406107287\

/home/wanxinli/Statistical-Fairness-Test/functions.py:52: RuntimeWarning: overflow encountered in exp
  gamma * lambd / (1 + np.exp(gamma * lambd * LA.norm(beta, ord=2) ** 2 * k - beta.T @ x_hat))) # objective function


gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.012626262626262628, beta.T @ x_hat is 15.917721604688026, the term within exp is: -754.9902351511721\gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.012626262626262628, beta.T @ x_hat is 15.9474464733816, the term within exp is: -755.0199600198656\gamma is -1.1803398874989481, lambd is -7.4074074074074066, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.011363636363636364, beta.T @ x_hat is 15.922046380996942, the term within exp is: 723.1504671654872\



/home/wanxinli/Statistical-Fairness-Test/functions.py:52: RuntimeWarning: overflow encountered in exp
  gamma * lambd / (1 + np.exp(gamma * lambd * LA.norm(beta, ord=2) ** 2 * k - beta.T @ x_hat))) # objective function


gamma is -1.1803398874989481, lambd is -7.4074074074074066, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.012626262626262628, beta.T @ x_hat is 15.923879385681053, the term within exp is: 805.2678023326346\
gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.012626262626262628, beta.T @ x_hat is 15.9191337022427, the term within exp is: -754.9916472487267\
gamma is -1.1803398874989481, lambd is -7.4074074074074066, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.012626262626262628, beta.T @ x_hat is 15.901742865877472, the term within exp is: 805.2899388524381\
gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.012626262626262628, beta.T @ x_hat is 15.912197762919966, the term within exp is: -754.984711309404\gamma is -1.1803398874989481, lambd is 6.666666666666666, LA.norm(beta, ord=2) is 86.24783536216124, k is 0.012626262626262628, beta.T @ x_hat is 15.8700901482372

In [ ]:
print(dist)

NameError: name 'dist' is not defined